## XAFS Data Processing (plotly)

This example shows how to use Larch from Python/Jupyter for basic XAFS data processing:

  1. read data from an Athena Project File
  2. do XAFS background subtraction with autobk()
  3. do XAFS Fourier transform with xfft()

In this version, plotting will be done with Plotly, which will plot to the Jupyter notebook, and
should give more interactivity to the plots compared to inline matplotlib.

For more details on Larch, see https://xraypy.github.io/xraylarch/xafs/

But first an important note on using Plotly with Jupyter:  
    Currently (December 2023) Plotly with LateX / Mathjax rendering of plot labels 
    works only with notebook version 6

So we begin by checking the version of notebook, and then importing the larch plotly routines 

In [1]:
import notebook, packaging.version
print(notebook.__version__)

from larch.plot.plotly_xafsplots import (plot_mu, plot_bkg, plot_chik, plot_chir, plot_chifit, 
                                         multi_plot, plotlabels, set_label_weight)

7.4.7


OK, with that, We'll start with reading and exploring the Athena Project File:

In [2]:
from larch.io import read_athena
project = read_athena('../xafsdata/fe_athena.prj')

In [3]:
for name, group in project.groups.items():
    print(name, group)

fe2o3_rt1_xmu <Group '0x1817c2ad0'>
fe3c_rt_xdi <Group '0x1817c2fd0'>
feo_rt1_xmu <Group '0x1817c34d0'>


> **NOTE** For additional features of working with Athena project files -> [IO_Athena_project.ipynb](./IO_Athena_project.ipynb) 

Let's get that data group for Fe2O3 and look at what that Group object contains

In [4]:
fe2o3 = project.fe2o3_rt1_xmu
for attr in dir(fe2o3):
    print(attr, type(getattr(fe2o3, attr)))

athena_id <class 'str'>
athena_params <class 'larch.symboltable.Group'>
atsym <class 'str'>
callargs <class 'larch.symboltable.Group'>
d2mude <class 'numpy.ndarray'>
datatype <class 'str'>
dmude <class 'numpy.ndarray'>
e0 <class 'float'>
edge <class 'str'>
edge_step <class 'float'>
edge_step_poly <class 'float'>
energy <class 'numpy.ndarray'>
energy_shift <class 'float'>
epsk <class 'float'>
epsr <class 'float'>
filename <class 'str'>
flat <class 'numpy.ndarray'>
i0 <class 'numpy.ndarray'>
journal <class 'larch.larchlib.Journal'>
label <class 'str'>
mu <class 'numpy.ndarray'>
norm <class 'numpy.ndarray'>
norm_poly <class 'numpy.ndarray'>
plot_xlabel <class 'str'>
plot_ylabel <class 'str'>
post_edge <class 'numpy.ndarray'>
pre_edge <class 'numpy.ndarray'>
pre_edge_details <class 'larch.symboltable.Group'>
sel <class 'int'>
signal <class 'numpy.ndarray'>
xdat <class 'numpy.ndarray'>
ydat <class 'numpy.ndarray'>
yerr <class 'float'>


There are several components to this Group of data.  We note that there are arrays for `energy` and `mu`.   These are what represent $\mu(E)$.

----------

To plot this data with plotly, we will use a simple Larch wrapper around plotly that gives a plot() function like the wx version.

In [5]:
from larch.plot.plotly_xafsplots import plot, multi_plot

plot_mu(fe2o3, show_norm=True)

Now let's remove the XAFS background and extract the EXAFS $\chi(k)$.   We'll use the `autobk()` function (https://xraypy.github.io/xraylarch/xafs/autobk.html)

Note that we pass in the *Group* as the first argument.  The `autobk()` function will add several attributes to this Group -- it will be the container for this dataset.

In [6]:
from larch.xafs import autobk
autobk(fe2o3, rbkg=0.9, kweight=2)

plot_bkg(fe2o3, norm=True)           

We'll also want to plot $\chi(k)$.  Note that, at this writing, using latex strings with Plotly in Jupyter is not working..

In [7]:
from larch.xafs import autobk
autobk(fe2o3, rbkg=0.85, kweight=2)

plot_chik(fe2o3, kweight=0)

but we usually want to plot $k$-weighted $\chi(k)$:

In [8]:
plot_chik(fe2o3, kweight=2)

and now we will do the Fourier transform from $\chi(k)$ to $\chi(R)$, using the `xftf()` (XAFS Fourier Transform Forward) function.  We'll do the Fourier transform on $k^2$ weighted $\chi(k)$ with a Kaiser-Bessel Window. 

In [9]:
from larch.xafs import xftf
xftf(fe2o3, kweight=2, kmin=2, kmax=13.0, dk=5, kwindow='Kaiser-Bessel')

plot_chik(fe2o3, kweight=2, show_window=True)

and we can show that data as $\chi(R)$:

In [10]:
plot_chir(fe2o3, show_real=True)

let's try another dataset with all the processing and plotting at once:

In [11]:
feo = project.feo_rt1_xmu

autobk(feo, rbkg=0.9, kweight=2)
xftf(feo, kweight=2, kmin=2, kmax=13.0, dk=5, kwindow='Kaiser-Bessel')

plot_chik(feo, kweight=2)
plot_chir(feo)